In [1]:
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim

Requirement already up-to-date: tensorflow==2.* in /usr/local/lib/python3.6/dist-packages (2.3.0rc2)


In [2]:
!pip install pycocotools


In [3]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [4]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [5]:
%%bash 
cd models/research
pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1304063 sha256=3a9f2fb3dc93d0078ffe4d652e5f5954f2c25871b4f13e279bc3a1fb2f83c1e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-hlerrtln/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection
  Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [6]:
import numpy as np
import os
import six
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib
import json
import datetime
import matplotlib.pyplot as plt

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [7]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils

In [8]:
utils_ops.tf = tf.compat.v1
tf.gfile = tf.io.gfile

In [9]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name,
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"
  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [10]:
PATH_TO_LABELS = 'models/research/object_detection/data/snapshot_serengeti_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=False)

In [11]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images/snapshot_serengeti')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpeg")))
TEST_IMAGE_PATHS

[PosixPath('models/research/object_detection/test_images/snapshot_serengeti/S1_E03_R3_PICT0038.jpeg'),
 PosixPath('models/research/object_detection/test_images/snapshot_serengeti/S1_E03_R3_PICT0039.jpeg'),
 PosixPath('models/research/object_detection/test_images/snapshot_serengeti/S1_E03_R3_PICT0040.jpeg'),
 PosixPath('models/research/object_detection/test_images/snapshot_serengeti/S1_E03_R3_PICT0041.jpeg')]

In [12]:
test_data_json = 'models/research/object_detection/test_images/snapshot_serengeti/context_rcnn_demo_metadata.json'
with open(test_data_json, 'r') as f:
  test_metadata = json.load(f)

In [13]:
image_id_to_datetime = {im['id']:im['date_captured'] for im in test_metadata['images']}
image_path_to_id = {im['file_name']: im['id'] 
                    for im in test_metadata['images']}
image_path_to_id

{'models/research/object_detection/test_images/snapshot_serengeti/S1_E03_R3_PICT0038.jpeg': 'S1/E03/E03_R3/S1_E03_R3_PICT0038',
 'models/research/object_detection/test_images/snapshot_serengeti/S1_E03_R3_PICT0039.jpeg': 'S1/E03/E03_R3/S1_E03_R3_PICT0039',
 'models/research/object_detection/test_images/snapshot_serengeti/S1_E03_R3_PICT0040.jpeg': 'S1/E03/E03_R3/S1_E03_R3_PICT0040',
 'models/research/object_detection/test_images/snapshot_serengeti/S1_E03_R3_PICT0041.jpeg': 'S1/E03/E03_R3/S1_E03_R3_PICT0041'}

In [14]:
faster_rcnn_model_name = 'faster_rcnn_resnet101_snapshot_serengeti_2020_06_10'
faster_rcnn_model = load_model(faster_rcnn_model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [15]:
faster_rcnn_model.inputs


[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]

In [16]:
faster_rcnn_model.output_dtypes


{'detection_boxes': tf.float32,
 'detection_classes': tf.float32,
 'detection_features': tf.float32,
 'detection_multiclass_scores': tf.float32,
 'detection_scores': tf.float32,
 'num_detections': tf.float32,
 'raw_detection_boxes': tf.float32,
 'raw_detection_scores': tf.float32}

In [17]:
faster_rcnn_model.output_shapes


{'detection_boxes': TensorShape([None, 300, 4]),
 'detection_classes': TensorShape([None, 300]),
 'detection_features': TensorShape([None, None, None, None, None]),
 'detection_multiclass_scores': TensorShape([None, 300, 49]),
 'detection_scores': TensorShape([None, 300]),
 'num_detections': TensorShape([None]),
 'raw_detection_boxes': TensorShape([None, 300, 4]),
 'raw_detection_scores': TensorShape([None, 300, 49])}

In [18]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)
  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_dets = output_dict.pop('num_detections')
  num_detections = int(num_dets)
  for key,value in output_dict.items():
    output_dict[key] = value[0, :num_detections].numpy() 
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(
      np.int64)
  return output_dict


In [19]:
def embed_date_captured(date_captured):
   embedded_date_captured = []
   month_max = 12.0
   day_max = 31.0
   hour_max = 24.0
   minute_max = 60.0
   min_year = 1990.0
   max_year = 2030.0

   year = (date_captured.year-min_year)/float(max_year-min_year)
   embedded_date_captured.append(year)

   month = (date_captured.month-1)/month_max
   embedded_date_captured.append(month)

   day = (date_captured.day-1)/day_max
   embedded_date_captured.append(day)

   hour = date_captured.hour/hour_max
   embedded_date_captured.append(hour)

   minute = date_captured.minute/minute_max
   embedded_date_captured.append(minute)

   return np.asarray(embedded_date_captured)


In [20]:
def embed_position_and_size(box):
   ymin = box[0]
   xmin = box[1]
   ymax = box[2]
   xmax = box[3]
   w = xmax - xmin
   h = ymax - ymin
   x = xmin + w / 2.0
   y = ymin + h / 2.0
   return np.asarray([x, y, w, h])


In [21]:
def get_context_feature_embedding(date_captured, detection_boxes,
                                  detection_features, detection_scores):
   date_captured = datetime.datetime.strptime(date_captured,'%Y-%m-%d %H:%M:%S')
   temporal_embedding = embed_date_captured(date_captured)
   embedding = detection_features[0]
   pooled_embedding = np.mean(np.mean(embedding, axis=1), axis=0)
   box = detection_boxes[0]
   position_embedding = embed_position_and_size(box)
   bb_embedding = np.concatenate((pooled_embedding, position_embedding))
   embedding = np.expand_dims(np.concatenate((bb_embedding,temporal_embedding)),
                             axis=0)
   score = detection_scores[0]
   return embedding, score

In [22]:
def run_inference(model, image_path, date_captured, resize_image=True):
   with open(image_path,'rb') as f:
    image = Image.open(f)
    if resize_image:
      image.thumbnail((640,640),Image.ANTIALIAS)
    image_np = np.array(image)

  # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)

   context_feature, score = get_context_feature_embedding(
      date_captured, output_dict['detection_boxes'],
      output_dict['detection_features'], output_dict['detection_scores'])
   return context_feature, score, output_dict


 

In [23]:
context_features = []
scores = []
faster_rcnn_results = {}
for image_path in TEST_IMAGE_PATHS:
  image_id = image_path_to_id[str(image_path)]
  date_captured = image_id_to_datetime[image_id]
  context_feature, score, results = run_inference(
      faster_rcnn_model, image_path, date_captured)
  faster_rcnn_results[image_id] = results
  context_features.append(context_feature)
  scores.append(score)

# Concatenate all extracted context embeddings into a contextual memory bank.
context_features_matrix = np.concatenate(context_features, axis=0)

In [ ]:
context_rcnn_model_name = 'context_rcnn_resnet101_snapshot_serengeti_2020_06_10'
context_rcnn_model = load_model(context_rcnn_model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
context_padding_size = 2000


In [ ]:
context_rcnn_model.inputs


In [ ]:
context_rcnn_model.output_dtypes


In [ ]:
context_rcnn_model.output_shapes


In [ ]:
def run_context_rcnn_inference_for_single_image(
    model, image, context_features, context_padding_size):
    image = np.asarray(image)
    image_tensor = tf.convert_to_tensor(
      image, name='image_tensor')[tf.newaxis,...]
    context_features = np.asarray(context_features)
    valid_context_size = context_features.shape[0]
    valid_context_size_tensor = tf.convert_to_tensor(
      valid_context_size, name='valid_context_size')[tf.newaxis,...]
    padded_context_features = np.pad(
      context_features,
      ((0,context_padding_size-valid_context_size),(0,0)), mode='constant')
    padded_context_features_tensor = tf.convert_to_tensor(
      padded_context_features,
      name='context_features',
      dtype=tf.float32)[tf.newaxis,...]
    output_dict = model(
      inputs=image_tensor,
      context_features=padded_context_features_tensor,
      valid_context_size=valid_context_size_tensor)
  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
    num_dets = output_dict.pop('num_detections')
    num_detections = int(num_dets)
    for key,value in output_dict.items():
      output_dict[key] = value[0, :num_detections].numpy() 
    output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    return output_dict
  




In [ ]:

def show_context_rcnn_inference(
    model, image_path, context_features, faster_rcnn_output_dict,
    context_padding_size, resize_image=True):
  """Runs inference over a single input image and visualizes Faster R-CNN vs. 
  Context R-CNN results.

  Args:
    model: A tensorflow saved_model object.
    image_path: Absolute path to the input image.
    context_features: A numpy float32 contextual memory bank of shape 
      [num_context_examples, 2057]
    faster_rcnn_output_dict: The output_dict corresponding to this input image
      from the single-frame Faster R-CNN model, which was previously used to
      build the memory bank.
    context_padding_size: The amount of expected padding in the contextual
      memory bank, defined in the Context R-CNN config as 
      max_num_context_features.
    resize_image: Whether to resize the input image before running inference.

  Returns:
    context_rcnn_image_np: Numpy image array showing Context R-CNN Results.
    faster_rcnn_image_np: Numpy image array showing Faster R-CNN Results.
  """

  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  with open(image_path,'rb') as f:
    image = Image.open(f)
    if resize_image:
      image.thumbnail((640,640),Image.ANTIALIAS)
    image_np = np.array(image)
    image.thumbnail((400,400),Image.ANTIALIAS)
    context_rcnn_image_np = np.array(image)
    
  faster_rcnn_image_np = np.copy(context_rcnn_image_np)

  # Actual detection.
  output_dict = run_context_rcnn_inference_for_single_image(
      model, image_np, context_features, context_padding_size)

  # Visualization of the results of a context_rcnn detection.
  vis_utils.visualize_boxes_and_labels_on_image_array(
      context_rcnn_image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      line_thickness=2)
  
  # Visualization of the results of a faster_rcnn detection.
  vis_utils.visualize_boxes_and_labels_on_image_array(
      faster_rcnn_image_np,
      faster_rcnn_output_dict['detection_boxes'],
      faster_rcnn_output_dict['detection_classes'],
      faster_rcnn_output_dict['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      line_thickness=2)
  return context_rcnn_image_np, faster_rcnn_image_np
 

In [ ]:
%matplotlib inline
plt.rcParams['axes.grid'] = False
plt.rcParams['xtick.labelsize'] = False
plt.rcParams['ytick.labelsize'] = False
plt.rcParams['xtick.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['ytick.right'] = False
plt.rcParams['figure.figsize'] = [15,10]

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  image_id = image_path_to_id[str(image_path)]
  faster_rcnn_output_dict = faster_rcnn_results[image_id]
  context_rcnn_image, faster_rcnn_image = show_context_rcnn_inference(
      context_rcnn_model, image_path, context_features_matrix,
      faster_rcnn_output_dict, context_padding_size)
  plt.subplot(1,2,1)
  plt.imshow(faster_rcnn_image)
  plt.title('Faster R-CNN')
  plt.subplot(1,2,2)
  plt.imshow(context_rcnn_image)
  plt.title('Context R-CNN')
  plt.show()
